In [1]:
#import libraries

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
plt.switch_backend('agg')
from mpl_toolkits.mplot3d import Axes3D
import neurokit2 as nk
import random
path = "/scratch/alim/overnight_validation/MUSE-PSG/"
import os
import random
import pickle
import heartpy as hp
apnea_labels = ["Mixed Apnea", "Central Apnea", "Obstructive Apnea"]

In [6]:
MUSE_SAMPLING_RATE = 64
apnea_labels = ["Mixed Apnea", "Central Apnea", "Obstructive Apnea"]

'''
THE FOLLOWING LENGTH OF CODE LINES 17-83 PROCESSES FILES INTO A USEABLE PATIENT_DATA_DIRECTORY
'''
def read_files_in_directory(directory_path):
    try:
        #print("Finding all MUSE-PSG files")
        # Get a list of all files in the directory
        file_list = sorted([entry.name for entry in os.scandir(directory_path) if
                    entry.is_file() and any(keyword in entry.name for keyword in ['ppg','events'])])[550:560]


                     
        # Read the contents of each file
        #print("Findind all file names completed")

        return file_list
    except Exception as e:
        print(f"Error reading files: {e}")

# Example usage:
directory_path = "/scratch/alim/overnight_validation/MUSE-PSG"
file_name_list = read_files_in_directory(directory_path)

patient_dictionary = {}

#print("Organizing into dictionary")

for filename in file_name_list:
    parts = filename.rsplit('_',1)
    patient_name = parts[0]
    
    if patient_name in patient_dictionary:
        patient_dictionary[patient_name].append(filename)

    else:
        patient_dictionary[patient_name] = [filename]

#print("Finished Organizing into dicitonary")

#print("Short Preview of the Dictionary:")

def read_csv_to_dataframe(file_path):
    try:
        # Read the CSV file into a Pandas DataFrame
        dataframe = pd.read_csv(file_path)

        # Return the DataFrame
        return dataframe

    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return None
    
patient_data_dictionary = {}
#read the csv for all patient files
for patient in patient_dictionary:
    files = patient_dictionary[patient]
    if len(files) != 2:
        continue
    patient_data_dictionary[patient] = {'events': read_csv_to_dataframe(path + files[0]),'ppg':read_csv_to_dataframe(path + files[1])}



#Change the time from unix format to regular date

for patient in patient_data_dictionary:

    patient_data_dictionary[patient]['ppg']['ts-datetime'] = pd.to_datetime(patient_data_dictionary[patient]['ppg']['ts'], unit='s')
    patient_data_dictionary[patient]['events']['start-datetime'] = pd.to_datetime(patient_data_dictionary[patient]['events']['start'], unit='s')
    patient_data_dictionary[patient]['events']['end-datetime'] = patient_data_dictionary[patient]['events']['start-datetime'] + pd.to_timedelta(patient_data_dictionary[patient]['events']['duration'], unit='s')

In [14]:
def getIndices(df):
    return df.index.values

def flattenedList(list_of_lists):
    flattened_list = [item for sublist in list_of_lists for item in sublist]
    return flattened_list

def consecutive_lists(sorted_list):
    result = []
    current_list = []

    for num in sorted_list:
        if not current_list or num == current_list[-1] + 1:
            current_list.append(num)
        else:
            result.append(current_list)
            current_list = [num]

    if current_list:
        result.append(current_list)

    return result

def randomlySampleList(lst,k):
    try:
        return random.sample(lst,k=k)
    
    except:
        return lst
    
def randomlySamplePpg_Df(df:pd.DataFrame,time:int) -> pd.DataFrame:
    try:
        first = df['ts-datetime'].values[0]
        last = df['ts-datetime'].values[-1]

        random_start_time = pd.to_datetime(random.uniform(first, last-pd.to_timedelta(time,unit = 's')), unit='s')
        window = df[(df['ts-datetime']>= random_start_time) & (df['ts-datetime']<= random_start_time + pd.to_timedelta(10,unit = 's'))]
        return window
    
    except:
        return df
    



In [4]:
matrix = np.empty((0,17))
sample_time = 10
apnea_dfs = []
regular_dfs = []

for patient in patient_data_dictionary:
    print(f"Processing Patient {patient}")
    allApneaPpg_dfs = []
    allEventPpg_dfs = []
    allNonEventPpg_dfs = []

    dictionary = patient_data_dictionary[patient]
    ppg = dictionary['ppg']
    #normalize teh data
    ppg['ch2'] = (ppg['ch2'] - ppg['ch2'].min(axis = 0))/(ppg['ch2'].max(axis=0)-ppg['ch2'].min(axis=0))
    events = dictionary['events']

    for index,event in events.iterrows():
        inEventCondition = (ppg['ts-datetime'] <= event['end-datetime']) & (ppg ['ts-datetime'] >= event['start-datetime'])
        eventPpg = ppg[inEventCondition]

        if event['name'] in apnea_labels:
            allApneaPpg_dfs.append(eventPpg)
        #eventPpg = randomlySamplePpg_Df(eventPpg,sample_time)
        allEventPpg_dfs.append(eventPpg)
    apnea_dfs+= allApneaPpg_dfs
    allEventPpg_Indices = sorted(list(set(flattenedList(list(map(getIndices,allEventPpg_dfs))))))
    
    nonEventPpg_Indices = ppg[~ppg.index.isin(allEventPpg_Indices)].index.values

    sample_size = len(allApneaPpg_dfs)
    list_Of_NonEventPpgIndices_Lists = consecutive_lists(nonEventPpg_Indices)

    randomlySampledNonEvents = randomlySampleList(list_Of_NonEventPpgIndices_Lists,sample_size)
    for indices in randomlySampledNonEvents:
        nonEventPpg = ppg.iloc[indices]
        nonEventPpg = randomlySamplePpg_Df(nonEventPpg,sample_time)
        allNonEventPpg_dfs.append(nonEventPpg)

    regular_dfs += allNonEventPpg_dfs
    print(f"Processed {len(allApneaPpg_dfs)} Apnea Events")
    print(f"Processed {len(randomlySampledNonEvents)} non-Events")
    
print(len(apnea_dfs))
print(len(regular_dfs))
 

Processing Patient 2023-03-09T214505-0500_6001-PDBR-6323


NameError: name 'flattenedList' is not defined

In [19]:
def extractFeatures(ppg,sample_rate,label:int):
    ppg = hp.filter_signal(ppg,cutoff = [0.7,3.5],sample_rate = 64, order = 3, filtertype = 'bandpass')
    wd,m = hp.process(ppg,sample_rate = sample_rate)

    #['bpm', 'ibi', 'sdnn', 'sdsd', 'rmssd', 'pnn20', 'pnn50', 'hr_mad', 'sd1', 'sd2', 's', 'sd1/sd2', 'breathingrate']
# Basic metrics
    heart_rate = m['bpm']  # Heart Rate in BPM
    ibi = m['ibi']  # Average Interbeat Interval
    sdnn = m['sdnn']  # Standard Deviation of NN intervals
    sdsd = m['sdsd']  # Standard Deviation of successive differences between adjacent NN intervals

    # HRV metrics
    rmssd = m['rmssd']  # Root Mean Square of Successive Differences
    pnn50 = m['pnn50']  # proportion of differences between adjacent NN intervals that are greater than 50ms

    std = np.std(ppg)
    mean = np.mean(ppg)
    var = np.var(ppg)


    feature_row = np.array([heart_rate,ibi,sdnn,sdsd,rmssd,m['pnn20'],pnn50,m['hr_mad'],m['sd1'],m['sd2'],m['s'],m['sd1/sd2'],m['breathingrate'],std,mean,var,label])
    return feature_row

In [20]:
matrix = np.empty((0,17))

for ppg in apnea_dfs:
    signal = ppg['ch2']
    features =  extractFeatures(ppg,MUSE_SAMPLING_RATE,1)
    matrix = np.vstack([matrix,features])


for ppg in regular_dfs:
    signal = ppg['ch2']
    features =  extractFeatures(ppg,MUSE_SAMPLING_RATE,1)
    matrix = np.vstack([matrix,features])


    

ValueError: The length of the input vector x must be greater than padlen, which is 21.

In [56]:
def check_nan_or_masked(data_dict):
    for value in data_dict.values():
        if value == "masked" or pd.isna(value):
            return True
    return False

In [7]:

column_names = ['patient_Id','bpm', 'ibi', 'sdnn', 'sdsd', 'rmssd', 'pnn20', 'pnn50', 'hr_mad', 'sd1', 'sd2', 's', 'sd1/sd2', 'breathingrate','std','mean','var','apnea']
return_Df = pd.DataFrame(columns=column_names)


sample_rate = 64
patient_Ids = []
dummy = patient_data_dictionary
for patient in dummy:
    apnea_matrix = pd.DataFrame(columns = column_names)
    non_apnea_matrix = pd.DataFrame(columns = column_names)
    events = dummy[patient]['events']
    ppg_signal = dummy[patient]['ppg']
    apnea = events[events['name'].isin(apnea_labels)]
    
    ppg_signal.set_index('ts-datetime', inplace=True)

    # Define the frequency for the intervals (e.g., '10S' for 10-second intervals)
    freq = '10S'

    # Group the DataFrame by the specified frequency
    ppg_10s_intervals = ppg_signal.groupby(pd.Grouper(freq=freq))

    # Now ppg_10s_intervals contains the DataFrame divided into 10-second intervals
    # You can access individual groups using the get_group method
    for interval, data in ppg_10s_intervals:
        label = 0

        ppg = data
        overlapping_apnea = apnea[(apnea['start-datetime'] <= interval  + pd.to_timedelta(10,unit = 's')) & (apnea['end-datetime'] >=interval)]
        if not ppg['ch2'].empty and (ppg['ch2'].max(axis=0) - ppg['ch2'].min(axis=0)) != 0:
            ppg.loc[:, 'ch2'] = (ppg['ch2'] - ppg['ch2'].min(axis=0)) / (ppg['ch2'].max(axis=0) - ppg['ch2'].min(axis=0))
        plt.clf()
        plt.plot(ppg['ts'].values,ppg['ch2'].values)
        plt.show()
        ppg = ppg['ch2']
        isApnea = not overlapping_apnea.empty
        if isApnea:
                label = 1
        
        try:
            
            #filter data
            ppg = hp.filter_signal(ppg,cutoff = [0.7,3.5],sample_rate = 64, order = 3, filtertype = 'bandpass')
            wd,m = hp.process(ppg,sample_rate = sample_rate)

            #['bpm', 'ibi', 'sdnn', 'sdsd', 'rmssd', 'pnn20', 'pnn50', 'hr_mad', 'sd1', 'sd2', 's', 'sd1/sd2', 'breathingrate']
        # Basic metrics
            heart_rate = m['bpm']  # Heart Rate in BPM
            ibi = m['ibi']  # Average Interbeat Interval
            sdnn = m['sdnn']  # Standard Deviation of NN intervals
            sdsd = m['sdsd']  # Standard Deviation of successive differences between adjacent NN intervals

            # HRV metrics
            rmssd = m['rmssd']  # Root Mean Square of Successive Differences
            pnn50 = m['pnn50']  # proportion of differences between adjacent NN intervals that are greater than 50ms

            std = np.std(ppg)
            mean = np.mean(ppg)
            var = np.var(ppg)

            #['patient_Id','bpm', 'ibi', 'sdnn', 'sdsd', 'rmssd', 'pnn20', 'pnn50', 'hr_mad', 'sd1', 'sd2', 's', 'sd1_sd2_ratio', 'breathingrate','std','mean','var','apnea']
            #feature_row = [patient,heart_rate,ibi,sdnn,sdsd,rmssd,m['pnn20'],pnn50,m['hr_mad'],m['sd1'],m['sd2'],m['s'],m['sd1/sd2'],m['breathingrate'],std,mean,var,label]
            feature_row = {
                'patient_Id':patient,
                'std':std,
                'mean':mean,
                'var':var,
                'apnea':label
            }

            feature_row = {**m,**feature_row}
            feature_row = pd.DataFrame([feature_row])

            is_null = feature_row.isnull()

            # Check for 'masked' values
            is_masked = feature_row == 'masked'

            # Combine the checks
            either_condition = is_null | is_masked

            # Check if any value in the DataFrame is either NaN or 'masked'
            contains_nan_or_masked = either_condition.any().any()
            if not contains_nan_or_masked:
                #print(f"Successfully processed!")
                if isApnea:
                    apnea_matrix = pd.concat([apnea_matrix,feature_row])
            
                else:
                    non_apnea_matrix = pd.concat([non_apnea_matrix,feature_row])
            

        except hp.exceptions.BadSignalWarning as e:
            pass
            #print("Bad signal quality. Consider checking the data or further preprocessing.")
        break

        # except Exception as e:
        #     print(f"An error occurred: {e.__class__.__name__}")
        #     print(f"Error message: {e}")
    break
    return_Df = pd.concat([return_Df,apnea_matrix])
    non_apnea_matrix = non_apnea_matrix.sample(min(len(apnea_matrix),len(non_apnea_matrix)))
    return_Df = pd.concat([return_Df,non_apnea_matrix])
    print(f"apnea_matrix length: {apnea_matrix.shape[0]}, non_apnea_matrix length = {non_apnea_matrix.shape[0]}, matrix length = {return_Df.shape[0]}")


#
print(return_Df)
        








Empty DataFrame
Columns: [patient_Id, bpm, ibi, sdnn, sdsd, rmssd, pnn20, pnn50, hr_mad, sd1, sd2, s, sd1/sd2, breathingrate, std, mean, var, apnea]
Index: []


/tmp/ipykernel_59347/2705039232.py:93: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  non_apnea_matrix = pd.concat([non_apnea_matrix,feature_row])


In [13]:
return_Df.to_csv('./matrix.csv')

PLOTTING HEART_RATE



In [6]:
def plotFeatures(labels,features,fileName,featureName):
    plt.scatter(x = labels, y = features)
    plt.title(featureName)
    plt.ylabel(featureName)
    plt.xlabel('apnea = 1, non-apnea = 0')
    plt.savefig(fileName)
    plt.clf()


In [32]:
#plotting heart rate
labels = matrix[:,16]

featureNames = ['bpm', 'ibi', 'sdnn', 'sdsd', 'rmssd', 'pnn20', 'pnn50', 'hr_mad', 'sd1', 'sd2', 's', 'sd1_sd2_ratio', 'breathingrate','std','mean','var']

for i in range(len(featureNames)):
    featureName = featureNames[i]

    features = matrix[:,i]

    plotFeatures(labels,features,f"./visualizations/{featureName}.png",featureName)



In [31]:
pd.DataFrame(matrix).to_csv('./matrix.csv',index = True)